# Scenario 1: Input an image from the camera and describe

## Install the libraries



In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.7 

In [ ]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 64.5 MB/s eta 0:00:00


In [ ]:
pip install openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=59cf29d7d1711c16a0d4271d081f777391c21cbd0ae71859d5909c6504ff4c14
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


## Image description through audio prompt in English/Hindi

In [ ]:
import gradio as gr
import librosa
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, BitsAndBytesConfig, pipeline
import whisper
from PIL import Image
import os

# Global variables to store models and processors
wav2vec2_processor = None
wav2vec2_model = None
whisper_model = None
llava_pipe = None
translator = None

# Function to load models and processors
def load_models():
    global wav2vec2_processor, wav2vec2_model, whisper_model, llava_pipe, translator

    # Load Wav2Vec2 models for Hindi
    wav2vec2_processor = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
    wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")

    # Load Hindi to English translator
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-hi-en")

    # Load Whisper model for English
    whisper_model = whisper.load_model("large")

    # Load LLaVA model
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )
    model_id = "llava-hf/llava-1.5-7b-hf"
    llava_pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

In [ ]:
load_models()

Some weights of the model checkpoint at Harveenchadha/vakyansh-wav2vec2-hindi-him-4200 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Harveenchadha/vakyansh-wav2vec2-hindi-him-4200 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Function to transcribe Hindi audio
def transcribe_hindi(wav_file):
    audio_input, sample_rate = resample_audio(wav_file, target_sampling_rate=16000)
    input_values = wav2vec2_processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values
    logits = wav2vec2_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = wav2vec2_processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

# Function to resample audio to the target sampling rate
def resample_audio(wav_file, target_sampling_rate):
    audio_input, sample_rate = librosa.load(wav_file, sr=target_sampling_rate)
    return audio_input, sample_rate

# Function to transcribe English audio
def transcribe_english(wav_file):
    transcription = whisper_model.transcribe(wav_file)
    return transcription["text"]

# Function to translate Hindi to English
def translate_to_english(text):
    translation = translator(text)
    return translation[0]['translation_text']

# Function to run image-to-text LLaVA model
def run_llava_model(transcription, image_path):
    image = Image.open(image_path)
    final_prompt = f"USER: <image>{transcription}\nASSISTANT:"
    outputs = llava_pipe(image, prompt=final_prompt, generate_kwargs={"max_new_tokens": 200})
    return outputs[0]["generated_text"]

def process(audio_file_path, image_file_path, language):
    if language.lower() == "hindi":
        hindi_transcription = transcribe_hindi(audio_file_path)
        english_transcription = translate_to_english(hindi_transcription)
    elif language.lower() == "english":
        english_transcription = transcribe_english(audio_file_path)
    else:
        return "Unsupported language. Please enter either 'Hindi' or 'English'."

    llava_result = run_llava_model(english_transcription, image_file_path)
    return llava_result


In [ ]:
# Define the Gradio interface
interface = gr.Interface(
    fn=process,
    inputs=[
        gr.Audio(type="filepath", label="Upload Audio File"),
        gr.Image(type="filepath", label="Upload Image File"),
        gr.Textbox(label="Language (Hindi or English)")
    ],
    outputs=gr.Textbox(label="Output"),
    title="Image Description Through Audio Prompt",
    description="Upload an audio file in English or Hindi and an image file. The audio will be transcribed and/or translated and used as a prompt for the image description model."
)

In [ ]:
interface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e7ca69038c49425fe9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
